In [1]:
from fdlite import FaceDetection, FaceLandmark, FaceDetectionModel
from fdlite.render import Colors, detections_to_render_data, landmarks_to_render_data, render_to_image
from PIL import Image
import numpy as np
from fdlite.types import Landmark

In [2]:
from PIL import Image
import cv2

detect_faces = FaceDetection(model_type=FaceDetectionModel.FRONT_CAMERA)
detect_landmarks = FaceLandmark()
cap = cv2.VideoCapture(0)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [3]:
def pil_to_cv2(img):
    return np.asarray(img)

In [4]:
def cv2_to_pil(img):
    return Image.fromarray(img)

In [5]:
from fdlite.types import *

def modify_detection(detection, pbox, sizes):
    dims = np.asarray(detection.bbox.as_tuple)
    pbox = np.asarray(pbox)

    mx = np.sum(dims[::2]) / 2.0
    my = np.sum(dims[1::2]) / 2.0

    ht, hb = my - dims[1], dims[3] - my
    wt, wb = mx - dims[0], dims[2] - mx

    dims = np.asarray([mx, my, mx, my])
    dims += np.asarray([-wt, -hb, wt, wb]) * pbox

    dims[0] = max(0, dims[0])
    dims[1] = max(0, dims[1])
    dims[2] = min(sizes[0], dims[2])
    dims[3] = min(sizes[1], dims[3])

    return Detection(dims, detection.score)

In [6]:
def alter_detections(detections, pbox, sizes):
    modified_detections = [modify_detection(detection, pbox, sizes) for detection in detections]
    return modified_detections

In [7]:
def create_circuit(line):
    return list(zip(line[:-1], line[1:]))

In [8]:
interest_points = [2, 19, 20, 60, 94, 97, 99, 125, 141, 164, 167, 241, 242, 250, 326, 328, 354, 370, 393, 458, 461, 462]

In [9]:
v_curve = create_circuit([0, 1, 4, 6, 8, 10])
h_curve = create_circuit([227, 1, 447])


def process_landmarks_for_face(detection, image):
    width, height = image.size
    dims = np.asarray(detection.bbox.as_tuple)
    dims[::2] *= width
    dims[1::2] *= height

    cropped = image.crop(dims)
    landmarks = detect_landmarks(cropped)

    points = []

    for index, landmark in enumerate(landmarks):
        x, y, z = landmark.x * (dims[2] - dims[0]), landmark.y * (dims[3] - dims[1]), landmark.z * 1.0
        x += dims[0]
        y += dims[1]

        points.append([x, y, z])

        landmarks[index] = Landmark(x / width, y / height, z)

    # print(points)
    if len(points) <= 0: return image, np.asarray([0.0, 0.0])

    origin = np.asarray(points[0]) - np.asarray([0.0, 30.0, 0.0])
    points = np.asarray(points)

    dist = points - origin
    dist = np.sum(dist ** 2, axis=1)
    dist = np.sqrt(dist)

    # print(np.expand_dims(dist,axis=1).shape)
    points = np.hstack((points, np.expand_dims(dist, axis=1)))
    indices = [index if value[3] <= 30 else 0 for index, value in enumerate(points)]
    indices = list(filter(lambda v: v != 0, indices))

    #print(indices)

    action = np.asarray([0.0, 0.0])
    for index in interest_points:
        action += np.asarray(points[index][:2]) - origin[:2]
    action /= len(interest_points)

    magnitude = np.sqrt(np.sum(action ** 2, axis=0))
    action /= magnitude

    landmarks_data = landmarks_to_render_data(landmarks, v_curve + h_curve)
    render_to_image(landmarks_data, image)
    return image, action

In [10]:
def process_landmarks(image, detections):
    actions = []
    for detection in detections:
        image, action = process_landmarks_for_face(detection, image)
        actions += [action]
    return image, actions

In [11]:
def process_frame(frame):
    frame = cv2_to_pil(frame)

    # try:
    detections = detect_faces(frame)
    if (len(detections) <= 0):
        return pil_to_cv2(frame), []

    altered_detections = alter_detections(detections, (1.35, 2.5, 1.35, 2.3), frame.size)
    frame, actions = process_landmarks(frame, altered_detections)
    # except:
    #     actions = []
    #     pass

    frame = pil_to_cv2(frame)
    return frame, actions

In [12]:
car = [int(1080 / 2), int(1080 / 2)]


def process():
    while (True):
        ret, frame = cap.read()
        frame, actions = process_frame(frame)
        canvas = frame.copy()

        canvas.fill(255)
        print(actions)
        render_frame = frame.copy()[:,::-1]
        height = canvas.shape[0]
        width = canvas.shape[1]

        render_frame = render_frame[:, int(height / 2) - 160:int(height) + int(height / 2) - 160]
        canvas[:, height - 240:] = render_frame[:, :]

        cv2.rectangle(canvas, (0, 0), (canvas.shape[0], canvas.shape[0]), (255, 255, 255), -1)

        #overlay
        cv2.rectangle(canvas, (canvas.shape[0] - 200, canvas.shape[0] - 200), (canvas.shape[0], canvas.shape[0]),
                      (0, 0, 0), -1)

        cv2.rectangle(canvas, (int(car[0] - 30), int(car[1] - 30)), (int(car[0] + 30), int(car[1] + 30)), (55, 55, 55),
                      -1)

        origin = (canvas.shape[0] - 100, canvas.shape[0] - 100)
        cv2.ellipse(canvas, origin, (10, 10), 0, 0, 360, (255, 255, 255), -1)

        if len(actions) > 0:
            xmove, ymove = actions[0][0], actions[0][1]
            car[0] += xmove * 5.3
            car[1] += ymove * 5.3

            if car[0] < 0: car[0] = height - 30
            if car[0] > height: car[0] = 30
            if car[1] < 0: car[1] = height - 30
            if car[1] > height: car[1] = 30
            cv2.ellipse(canvas, (int(origin[0] + 20 * xmove), int(origin[1] + 20 * ymove)), (10, 10), 0, 0, 360,
                        (255, 255, 255), -1)

        cv2.imshow('frame', canvas)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

In [13]:
process()

[]
[array([-0.75285885, -0.65818201])]
[array([-0.65625178, -0.75454198])]
[array([-0.66975185, -0.74258498])]
[array([-0.71151747, -0.7026684 ])]
[array([-0.64630302, -0.76308087])]
[array([-0.65921712, -0.75195265])]
[array([-0.70099299, -0.71316816])]
[array([-0.62857156, -0.77775176])]
[array([-0.57913171, -0.81523399])]
[array([-0.57001919, -0.82163138])]
[array([-0.60724984, -0.79451094])]
[array([-0.72905155, -0.68445879])]
[array([-0.70357733, -0.7106187 ])]
[array([-0.752908  , -0.65812578]), array([0., 0.])]
[array([-0.67791651, -0.7351389 ]), array([0., 0.])]
[array([-0.75518594, -0.65551064]), array([-0.52811074, -0.84917551])]
[array([-0.76166452, -0.64797157]), array([-0.48919084, -0.87217677])]
[array([-0.79145797, -0.6112236 ]), array([-0.53788926, -0.84301551])]
[array([-0.7752774 , -0.63162089])]
[array([-0.7683862 , -0.63998645])]
[array([-0.61618298, -0.78760303])]
[array([-0.78132326, -0.62412655])]
[array([-0.80763491, -0.58968284])]
[array([-0.84002411, -0.542549

KeyboardInterrupt: 

In [ ]:
cap.release()